In [42]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torch.optim import Adam
from torch.nn.utils import clip_grad_norm_
import numpy as np
import pandas as pd

In [38]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [19]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.linear_subjective = nn.Linear(768,10)  
        self.linear_medhx = nn.Linear(768, 10)
        self.combined = nn.Linear(30, 10)
        self.out = nn.Linear(10,4)

    def forward(self, subj,medhx,cont):
        nlp1 = F.relu(self.linear_subjective(subj))
        print (nlp1.shape)
        nlp2 = F.relu(self.linear_medhx(medhx))
        print (nlp2.shape)
        combined = torch.cat((nlp1,nlp2,cont), axis = 1)
        print (combined.shape)
        x = self.combined(combined)
        return x



In [20]:

net = Net()
print(net)

Net(
  (linear_subjective): Linear(in_features=768, out_features=10, bias=True)
  (linear_medhx): Linear(in_features=768, out_features=10, bias=True)
  (combined): Linear(in_features=30, out_features=4, bias=True)
)


In [25]:
subj = torch.rand(16,768)
medhx = torch.rand(16,768)
cont = torch.rand(16,10)

In [26]:
subj.shape, medhx.shape, cont.shape

(torch.Size([16, 768]), torch.Size([16, 768]), torch.Size([16, 10]))

In [28]:
net(subj,medhx,cont)

torch.Size([16, 10])
torch.Size([16, 10])
torch.Size([16, 30])


tensor([[-0.1945, -0.1271,  0.0957, -0.0020],
        [-0.2033, -0.3106, -0.0561,  0.2557],
        [-0.1573, -0.2584, -0.0609,  0.0212],
        [-0.2167, -0.1463,  0.1489, -0.0350],
        [-0.0876, -0.2825,  0.0071, -0.0218],
        [-0.1567, -0.2244,  0.1035, -0.0768],
        [-0.2046, -0.1121,  0.1961,  0.1599],
        [-0.3198, -0.1004,  0.2665,  0.0888],
        [-0.2699, -0.1097,  0.1207, -0.0498],
        [-0.1291, -0.4233,  0.0331,  0.1110],
        [-0.1529, -0.2097,  0.0995, -0.2116],
        [-0.2219, -0.1162,  0.1114, -0.0412],
        [ 0.1215, -0.4643, -0.0387,  0.0072],
        [-0.2420, -0.0083,  0.1116, -0.0340],
        [-0.0671, -0.1996,  0.0701,  0.0099],
        [-0.2652, -0.2181,  0.0434,  0.0870]], grad_fn=<AddmmBackward>)

In [41]:
#hyperparameters
lr = 1e-3
optimizer = Adam(net.parameters(), lr = lr)
epochs = 3
loss_func = nn.CrossEntropyLoss()

In [ ]:
def train_model(model):
    for epoch_num in range(epochs):
    model.train()
    train_loss = 0
    for step_num, batch_data in enumerate(trainloader):
        
        cont_var, subj_notes, medhx, labels = tuple(t.to(device) for t in batch_data)
        
        optimzer.zero_grad()
        
        logits = model(cont_var, subj_notes, medhx)
        
        batch_loss = loss_func(logits, labels)
        
        train_loss += batch_loss.item()
        
        batch_loss.backward()
        

        clip_grad_norm_(parameters=model.parameters(), max_norm=1.0)
        
        optimizer.step()
        
        clear_output(wait=True)
        print('Epoch: ', epoch_num + 1)
        print("\r" + "{0}/{1} loss: {2} ".format(step_num, len(train_data) / BATCH_SIZE, train_loss / (step_num + 1)))
        

In [43]:
path = '/Users/jjaskolkambp/Desktop/machine learning/my_projects/ed-triage'
data_path = '/Users/jjaskolkambp/Desktop/machine learning/my_projects/data/ED triage project/egh'

In [46]:
embeds = pd.read_csv(data_path + '/embeds.csv', index_col = 0)
embeds.shape

(85153, 1)